In [ ]:
import numpy as np 
import pandas as pd
import SAGEnet.data 
import SAGEnet.tools
from SAGEnet.models import pSAGEnet
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import glob
import os

#### Notes:   
-- Because of data protections, the data we used in our analyses (ROSMAP and GTEx WGS and expression data) is not provided for use in this github. Please see the paper for specifics on how to request that data.   
-- Because of this, we instead provide toy data in input_data/example data to demonstrate required data formats for this tutorial.  
-- In addition, we do not include the hg38 genome in this github due to its size -- but you can go to input_data/ and run download_genome.sh to save a copy of the hg38 genome to use.  
-- Finally, before running this notebook, please follow the steps under "installation" in the main READme to install dependencies and the SAGEnet package.

## Initializing a PersonalGenomeDataset

#### First, define your paths for the hg38 genome, file that defines TSS locations for each gene, and VCF. We provide a toy VCF example file in input_data/example_data/:

In [ ]:
hg38_file_path='input_data/hg38.fa'
example_vcf_file_path='input_data/example_data/example_vcf.vcf.gz'
tss_data_path='input_data/gene-ids-and-positions.tsv'

#### Then load your data: 
-- example_individuals is a list of sample names, as they appear in the VCF. For this toy data, we include three sample individuals.   
-- example_expression_data is a DataFrame of expression data indexed by gene names, with sample names as columns.  For this toy data, we use a single gene, "ENSG00000013573".  
-- gene_meta_info is a DataFrame of gene metadata containing the columns "chr", "tss", and "strand". 

In [25]:
example_individuals = np.loadtxt('input_data/example_data/example_individuals.csv',delimiter=',',dtype=str)
example_individuals

array(['SAMPLE1', 'SAMPLE2', 'SAMPLE3'], dtype='<U7')

In [27]:
example_expression_data = pd.read_csv('input_data/example_data/example_expression.csv',index_col=0) 
example_expression_data

,SAMPLE1,SAMPLE2,SAMPLE3
ENSG00000013573,2.1,0.3,0.7


In [28]:
gene_meta_info = pd.read_csv(tss_data_path, sep='\t')
gene_meta_info

,gene_name,gene_id,chr_hg38,start_hg38,end_hg38,strand_hg38,tss_hg38,chr_hg19,tss_hg19,tss,chr,ensg,strand
0,DDX11L1,ENSG00000223972,1,11869,14409,+,11869,chr1,11869.0,11869,1,ENSG00000223972,+
1,WASH7P,ENSG00000227232,1,14404,29570,-,29570,chr1,29570.0,29570,1,ENSG00000227232,-
2,MIR6859-1,ENSG00000278267,1,17369,17436,-,17436,chr1,17436.0,17436,1,ENSG00000278267,-
3,MIR1302-2HG,ENSG00000243485,1,29554,31109,+,29554,chr1,29554.0,29554,1,ENSG00000243485,+
4,MIR1302-2,ENSG00000284332,1,30366,30503,+,30366,chr1,30366.0,30366,1,ENSG00000284332,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58297,AC240274.1,ENSG00000271254,KI270711.1,4612,29626,-,29626,NaN,NaN,29626,KI270711.1,ENSG00000271254,-
58298,U1,ENSG00000275405,KI270713.1,21861,22024,-,22024,NaN,NaN,22024,KI270713.1,ENSG00000275405,-
58299,U1,ENSG00000275987,KI270713.1,30437,30580,-,30580,NaN,NaN,30580,KI270713.1,ENSG00000275987,-
58300,AC213203.1,ENSG00000277475,KI270713.1,31698,32528,-,32528,NaN,NaN,32528,KI270713.1,ENSG00000277475,-


#### Select gene meta information for our example gene, "ENSG00000013573": 

In [29]:
example_gene_meta_info=gene_meta_info[gene_meta_info['gene_id']=='ENSG00000013573']
example_gene_meta_info

,gene_name,gene_id,chr_hg38,start_hg38,end_hg38,strand_hg38,tss_hg38,chr_hg19,tss_hg19,tss,chr,ensg,strand
36577,DDX11,ENSG00000013573,12,31073845,31104791,+,31073845,chr12,31226779.0,31073845,12,ENSG00000013573,+


#### Now that we have our example_expression_data, example_gene_meta_info, vcf_file_path (which contains variant info for our example gene), and hg38_file_path, we can initialize a basic PersonalGenomeDataset:  

Important paramters to change to suit your needs:   
-- "contig_prefix" gives the string before chromosome number in VCF file. Since our example VCF presents the chromosome as "chr12", we used the input contig_prefix='chr' to PersonalGenomeDataset. If the format were instead "12", we would use the input contig_prefix=''. Default: ''.   
-- "split_expr" controls whether expression is to be decomposed into mean and difference from mean. If True, idx 1 of the expression output represents difference from mean expression. If False, idx 1 of the expression output represents personal gene expression (used to train non-contrastive model). Default: True.   
-- "input_len" controls sequence length.   
-- "expr_data_zscore" determines whether (if "split_expr" is True), z-scores will be used as difference from mean. If not, the difference used will be simply personal expression - mean expression. Default: True.   
-- "only_snps" determine if only SNPs (or if variants of any length) are inserted into the personal genome (by default, all variants are inserted). Default: False.   
-- "expr_data" must be provided if you want to output personal expression values. If it is not provided, zeros will be used in place of expression values. This can be useful if you are just evaluating a model, not training. Default: None.   
-- "maf_threshold" determines the threshold MAF for a variant to be inserted. If less than zero, all variants will be inserted. Default: -1.   

To initialize a PersonalGenomeDataset to use to train a non-contrastive model, set split_expr=False. 

For more insight into parameters and how to adapt PersonalGenomeDataset, please see the class documentation.   

In [32]:
personal_dataset = SAGEnet.data.PersonalGenomeDataset(gene_metadata=example_gene_meta_info, vcf_file_path=example_vcf_file_path, hg38_file_path=hg38_file_path, sample_list=example_individuals, expr_data=example_expression_data,contig_prefix='chr')

#### As we can see, the number of datapoints in the PersonalGenomeDataset is equal to the number of genes x the number of individuals (here, 1x3). Each item in the PersonalGenomeDataset is a tuple containing: 
-- One-hot-encoded tensor of genomic sequence of shape [2,8,40000]. This contains personal genomic sequence from each haplotype (maternal: [0,:4,:], paternal: [0,4:,:]) and reference sequence ([1,:,:]).  
-- Expression tensor containing [mean expression, personal difference from mean expression]  
-- Gene index  
-- Sample index  

Gene index and sample index are used to get correlation (across gene and across individual) metrics at epoch ends. 

In [34]:
for item in personal_dataset:
    print('new datapoint:')
    print(item[0].shape)
    print(item[1])
    print(item[2])
    print(item[3])
    print('---')

new datapoint:
torch.Size([2, 8, 40000])
tensor([1.0333, 1.0667])
0
0
---
new datapoint:
torch.Size([2, 8, 40000])
tensor([ 1.0333, -0.7333])
0
1
---
new datapoint:
torch.Size([2, 8, 40000])
tensor([ 1.0333, -0.3333])
0
2
---


#### There should be no lag time when iterating through the PersonalGenomeDataset. 
#### The process for creating a ReferenceGenomeDataset or VariantDataset is very similar, please see the READme in SAGEnet/ and the class documentations for details. 

## Initializing a pSAGEnet model 

#### A pSAGEnet model with default parameters can be initialized using: 

In [20]:
my_model = pSAGEnet()

Important paramters to change to suit your needs:   
-- many paramters ("input_length", "first_layer_kernel_number", "int_layers_kernel_number", etc.) control aspects of model architecture. Please see class documentation for details.   
-- "lam_ref" and "lam_diff" control the weights on each componenent of the loss function. "lam_ref" controls the weight on the 0 idx output (mean) and "lam_diff" controls the weight on the 1 idx output (difference).   
-- "split_expr": if False, model "difference" output (idx 1) is predicted straight from personal sequence (no intermediate subtraction with reference).  

To train a "non-contrastive" model, set lam_ref=0, split_expr=False (along with setting split_expr=False in the PersonalGenomeDataset).

For more insight into parameters and how to adapt pSAGEnet, please see the class documentation.   

#### The process for initializing a rSAGEnet model is very similar, please see class documentation for details. 

## Training a pSAGEnet model

#### The full code required to train a pSAGEnet model can be found in script/train_mdoel/train_model.py. For simplified keys sections, see below. 

#### Load in your training and validation individuals. The train/validation splits we use for ROSMAP are found in input_data/individual_sets/. 

In [4]:
sub_data_dir='input_data/individual_sets/'
train_subs = np.loadtxt(f'{sub_data_dir}ROSMAP/train_subs.csv',delimiter=',',dtype=str)
val_subs = np.loadtxt(f'{sub_data_dir}ROSMAP/val_subs.csv',delimiter=',',dtype=str)

#### Each of these is just a list of individuals (with names that match what's found in the VCF & the expression data): 

In [6]:
print(train_subs.shape)
print(val_subs.shape)
print(val_subs[:5])

(689,)
(85,)
['P10315029_P10315029' 'P35286551_P35286551' 'P84642424_P84642424'
 'P50401390_P50401390' 'P20249897_P20249897']


#### Get list of genes to use in training and validation. In this example, let's use the top 1000 genes based on the predixcan ranking. 

In [ ]:
predixcan_res_path='results_data/predixcan/rosmap_pearson_corr.csv'
rand_genes=0 # set to 0 because we are using "top genes" not random genes
num_top_genes=1000 # based on predixcan ranking 
gene_idx_start=0 # use genes rank 0 -- rank 999 
gene_list = SAGEnet.tools.select_gene_set(predixcan_res_path=predixcan_res_path, rand_genes=rand_genes,num_genes=num_top_genes,gene_idx_start=gene_idx_start) 
print(gene_list.shape)

selecting genes from 0 to 1000
(1000,)


#### Split this list of genes into train and validation based on chromsome split (defined in SAGEnet.tools)

In [12]:
train_gene_list, val_gene_list , test_gene_list = SAGEnet.tools.get_train_val_test_genes(gene_list, tss_data_path=tss_data_path)
print(train_gene_list.shape)
print(val_gene_list.shape)
print(test_gene_list.shape)

selecting train/val/test gene sets based on chromosome split
(741,)
(118,)
(117,)


#### Get the TSS information associated with each gene set: 

In [14]:
gene_meta_info = pd.read_csv(tss_data_path, sep="\t")
train_gene_meta=gene_meta_info[gene_meta_info['ensg'].isin(train_gene_list)]
val_gene_meta=gene_meta_info[gene_meta_info['ensg'].isin(val_gene_list)]
train_gene_meta

,gene_name,gene_id,chr_hg38,start_hg38,end_hg38,strand_hg38,tss_hg38,chr_hg19,tss_hg19,tss,chr,ensg,strand
62,PERM1,ENSG00000187642,1,975204,982093,-,982093,chr1,917473.0,982093,1,ENSG00000187642,-
81,TNFRSF18,ENSG00000186891,1,1203508,1206691,-,1206691,chr1,1142071.0,1206691,1,ENSG00000186891,-
108,ANKRD65,ENSG00000235098,1,1418420,1421769,-,1421769,chr1,1357149.0,1421769,1,ENSG00000235098,-
113,ATAD3C,ENSG00000215915,1,1449689,1470158,+,1449689,chr1,1385069.0,1449689,1,ENSG00000215915,+
127,SLC35E2B,ENSG00000189339,1,1659529,1692728,-,1692728,chr1,1624167.0,1692728,1,ENSG00000189339,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46899,RNF166,ENSG00000158717,16,88696495,88706421,-,88706421,chr16,88772829.0,88706421,16,ENSG00000158717,-
46912,TRAPPC2L,ENSG00000167515,16,88856220,88862686,+,88856220,chr16,88922628.0,88856220,16,ENSG00000167515,+
46925,SLC22A31,ENSG00000259803,16,89195761,89201664,-,89201664,chr16,89268072.0,89201664,16,ENSG00000259803,-
46954,SPIRE2,ENSG00000204991,16,89818179,89871319,+,89818179,chr16,89884587.0,89818179,16,ENSG00000204991,+


#### Following the guidelines from earlier in the notebook, create a PersonalGenomeDataset and Dataloaders for train and validation: 
-- For data protection reasons described at the top of the notebook, these cells cannot be run because they need the actual ROSMAP vcf_file_path and expr_data, which we cannot add to the github.   
-- train_dataset is train individuals, train genes. val_subs_dataset is val individuals, train genes. val_genes_dataset is val individuals, val genes. 

In [ ]:
train_dataset = SAGEnet.data.PersonalGenomeDataset(gene_metadata=train_gene_meta, vcf_file_path=vcf_file_path, hg38_file_path=hg38_file_path, sample_list=train_subs, expr_data=expr_data)
val_subs_dataset = SAGEnet.data.PersonalGenomeDataset(gene_metadata=train_gene_meta, vcf_file_path=vcf_file_path, hg38_file_path=hg38_file_path, sample_list=val_subs, expr_data=expr_data)
val_genes_dataset = SAGEnet.data.PersonalGenomeDataset(gene_metadata=val_gene_meta, vcf_file_path=vcf_file_path, hg38_file_path=hg38_file_path, sample_list=val_subs, expr_data=expr_data)

train_dataloader = DataLoader(train_dataset,shuffle=True)
val_subs_dataloader = DataLoader(val_subs_dataset, shuffle=False)
val_genes_dataloader = DataLoader(val_genes_dataset,shuffle=False)

#### Set up for model training:

In [ ]:
model_save_dir='test_model_save_dir/'

val_dataloaders=[val_subs_dataloader,val_genes_dataloader]
es = EarlyStopping(monitor="train_gene_val_sub_diff_loss/dataloader_idx_0", patience=5,mode='min')
lr_monitor = LearningRateMonitor(logging_interval='epoch')

# used to save every model epoch 
all_epoch_checkpoint_callback = ModelCheckpoint(
    dirpath=model_save_dir,  
    filename="{epoch}",  
    save_top_k=-1,  
    every_n_epochs=1,
    save_last=False
)

# save last ckpt to be able to resume model training if job is killed 
last_checkpoint_callback = ModelCheckpoint(
    dirpath=model_save_dir,
    filename="last",     
    save_top_k=0,        
    every_n_train_steps=300,  
    save_last=True      
)

ckpt_list = [all_epoch_checkpoint_callback,last_checkpoint_callback]

#### Resume if an earlier training run was interrupted: 

In [ ]:
if glob.glob(os.path.join(model_save_dir, "*.ckpt"))!=[]:
    last_checkpoint = model_save_dir + "/last.ckpt"
else:
    last_checkpoint = None

#### Set up a trainer (in our case, using distributed training across multiple nodes):
-- To train on a single GPU, set device equal to the integer label of the GPU you want to use. You can also edit the "accelerator" argument to use with only CPU. 

In [ ]:
wandb_logger = WandbLogger(project='test_project_name', name='test_job_name', id='test_job_name', resume="allow")
device=None # to indicate mulit-node training 
num_nodes=8
max_epochs=10

trainer = pl.Trainer(
    accelerator="gpu", 
    devices=[int(device)] if device else 1, 
    num_nodes=num_nodes, 
    strategy="ddp" if not device else None, 
    callbacks=ckpt_list, 
    max_epochs=max_epochs, 
    benchmark=False, 
    profiler='simple', 
    gradient_clip_val=1, 
    logger=wandb_logger, 
    log_every_n_steps=10)

#### Optionally initialize p-SAGE-net from a pre-trained model (in our case, r-SAGE-net). 

In [ ]:
ref_model_ckpt_path='reference_model.ckpt'
print(f'loading rSAGEnet weights from {ref_model_ckpt_path} into pSAGEnet model')
my_model = SAGEnet.tools.init_model_from_ref(my_model, ref_model_ckpt_path)

#### Train! 

In [ ]:
if last_checkpoint is None:
    print('fitting model')
    trainer.fit(my_model, train_dataloader, val_dataloaders=val_dataloaders)
else:
    print(f'fitting model from ckpt={last_checkpoint}')
    trainer.fit(my_model, train_dataloader, val_dataloaders=val_dataloaders, ckpt_path=last_checkpoint)

#### For more examples, please see the READmes in each directory and the code documentation.